In [1]:
def Validation(n_fold,X):
    list_train_fold = []
    list_val_fold = []
    list_train = []
    Number = X.shape[0]//n_fold
    for i in range(X.shape[0]):
        list_train.append(i)
    
    for i in range(n_fold)[::-1]:
        list_val = []
        if i==n_fold-1:
            for j in range(Number*i,X.shape[0]):
                list_val.append(j)
                
            list_train_fold.append(np.setdiff1d(list_train,list_val))
            list_val_fold.append(list_val)
            
        
        if i != n_fold-1: 
            for j in range(Number*i,Number*(i+1)):
                list_val.append(j)
            list_train_fold.append(np.setdiff1d(list_train,list_val))
            list_val_fold.append(list_val)
        
    return list_train_fold, list_val_fold

In [2]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(0, '..')
from lib import models_update, graph, coarsening, utils
# modified
import tensorflow.compat.v1 as tf
import numpy as np
import time
import pandas as pd
%matplotlib inline
import random
import tensorflow as tf
tf.__version__


'1.15.0'

In [3]:
def data_simulation(exp, ppi,p):
#     KEGG = 238
    KEGG = 2
    dataset = {}
    all_nodes = exp.shape[1]
    dataset['exp'] = []
    dataset['L'] = []
    dataset['perm'] = []
    Pathway_Name = list(p)
    
        
    for i in range(len(Pathway_Name)):
        #node_num = random.randint(300, 400)
        #sample_nodes = random.sample(range(all_nodes), node_num)
        Node_Number = []
        Sub_Pathway = p[Pathway_Name[i]]['gene'].values()
        Numbering = list(Sub_Pathway)
        for a in range(len(Numbering)):
            for b in range(exp.shape[1]):
                if Numbering[a] == exp.columns[b]:
                    Node_Number.append(b)

        sample_ppi = ppi.iloc[Node_Number, Node_Number]
        sample_ppi = np.array(sample_ppi)
        print("PPI Shape :",len(Node_Number), sample_ppi.shape[0])
        A = coo_matrix(sample_ppi,dtype=np.float32)
        print("Number of Edge :",A.nnz//2, "//",i)
        
        graphs, perm = coarsening.coarsen(A, levels=4, self_connections=False)
        print(type(A))
        L = [graph.laplacian(A, normalized=True) for A in graphs]
#         print(len(L))
#         graph.plot_spectrum(L)
        dataset['exp'].append(exp.iloc[:, Node_Number].values)
        dataset['L'].append(L)
        dataset['perm'].append(perm)
    return dataset

In [4]:
import pickle
from scipy.sparse import coo_matrix
ppi = pd.read_csv("Table/PPI.csv")
#ppi = pd.read_csv("Data/L1000/groupPPI.csv")
ppi.drop(['string'], axis='columns', inplace=True)
#ppi.drop(['Unnamed: 0'], axis='columns', inplace=True)

exp = pd.read_csv("Table/EXP.csv")
#exp = pd.read_csv("Data/L1000/groupEXP.csv")
exp.drop(['Unnamed: 0'], axis='columns', inplace=True)
print(exp.shape)
print(ppi.shape)
with open('Table/Pathway.pkl','rb')as f:
    p = pickle.load(f)



(734, 7545)
(7545, 7545)


In [5]:
dataset = data_simulation(exp, ppi,p)


PPI Shape : 35 35
Number of Edge : 479 // 0
Layer 0: M_0 = |V| = 48 nodes (13 added),|E| = 479 edges
Layer 1: M_1 = |V| = 24 nodes (6 added),|E| = 148 edges
Layer 2: M_2 = |V| = 12 nodes (3 added),|E| = 36 edges
Layer 3: M_3 = |V| = 6 nodes (1 added),|E| = 10 edges
Layer 4: M_4 = |V| = 3 nodes (0 added),|E| = 3 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 17 17
Number of Edge : 129 // 1
Layer 0: M_0 = |V| = 32 nodes (15 added),|E| = 129 edges
Layer 1: M_1 = |V| = 16 nodes (7 added),|E| = 36 edges
Layer 2: M_2 = |V| = 8 nodes (3 added),|E| = 10 edges
Layer 3: M_3 = |V| = 4 nodes (1 added),|E| = 3 edges
Layer 4: M_4 = |V| = 2 nodes (0 added),|E| = 1 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 26 26
Number of Edge : 248 // 2
Layer 0: M_0 = |V| = 32 nodes (6 added),|E| = 248 edges
Layer 1: M_1 = |V| = 16 nodes (3 added),|E| = 77 edges
Layer 2: M_2 = |V| = 8 nodes (1 added),|E| = 21 edges
Layer 3: M_3 = |V| = 4 nodes (0 added),|E| = 6 edges
Layer 4: M_4 = |V| = 2 node

PPI Shape : 40 40
Number of Edge : 482 // 24
Layer 0: M_0 = |V| = 48 nodes (8 added),|E| = 482 edges
Layer 1: M_1 = |V| = 24 nodes (4 added),|E| = 162 edges
Layer 2: M_2 = |V| = 12 nodes (2 added),|E| = 44 edges
Layer 3: M_3 = |V| = 6 nodes (1 added),|E| = 10 edges
Layer 4: M_4 = |V| = 3 nodes (0 added),|E| = 3 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 19 19
Number of Edge : 114 // 25
Layer 0: M_0 = |V| = 32 nodes (13 added),|E| = 114 edges
Layer 1: M_1 = |V| = 16 nodes (6 added),|E| = 36 edges
Layer 2: M_2 = |V| = 8 nodes (2 added),|E| = 11 edges
Layer 3: M_3 = |V| = 4 nodes (1 added),|E| = 3 edges
Layer 4: M_4 = |V| = 2 nodes (0 added),|E| = 1 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 24 24
Number of Edge : 186 // 26
Layer 0: M_0 = |V| = 32 nodes (8 added),|E| = 186 edges
Layer 1: M_1 = |V| = 16 nodes (4 added),|E| = 61 edges
Layer 2: M_2 = |V| = 8 nodes (2 added),|E| = 15 edges
Layer 3: M_3 = |V| = 4 nodes (1 added),|E| = 3 edges
Layer 4: M_4 = |V| = 2 no

PPI Shape : 57 57
Number of Edge : 589 // 47
Layer 0: M_0 = |V| = 64 nodes (7 added),|E| = 589 edges
Layer 1: M_1 = |V| = 32 nodes (3 added),|E| = 259 edges
Layer 2: M_2 = |V| = 16 nodes (1 added),|E| = 96 edges
Layer 3: M_3 = |V| = 8 nodes (0 added),|E| = 28 edges
Layer 4: M_4 = |V| = 4 nodes (0 added),|E| = 6 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 32 32
Number of Edge : 226 // 48
Layer 0: M_0 = |V| = 32 nodes (0 added),|E| = 226 edges
Layer 1: M_1 = |V| = 16 nodes (0 added),|E| = 84 edges
Layer 2: M_2 = |V| = 8 nodes (0 added),|E| = 23 edges
Layer 3: M_3 = |V| = 4 nodes (0 added),|E| = 6 edges
Layer 4: M_4 = |V| = 2 nodes (0 added),|E| = 1 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 31 31
Number of Edge : 209 // 49
Layer 0: M_0 = |V| = 32 nodes (1 added),|E| = 209 edges
Layer 1: M_1 = |V| = 16 nodes (0 added),|E| = 81 edges
Layer 2: M_2 = |V| = 8 nodes (0 added),|E| = 24 edges
Layer 3: M_3 = |V| = 4 nodes (0 added),|E| = 6 edges
Layer 4: M_4 = |V| = 2 nod

PPI Shape : 46 46
Number of Edge : 607 // 70
Layer 0: M_0 = |V| = 48 nodes (2 added),|E| = 607 edges
Layer 1: M_1 = |V| = 24 nodes (1 added),|E| = 223 edges
Layer 2: M_2 = |V| = 12 nodes (0 added),|E| = 65 edges
Layer 3: M_3 = |V| = 6 nodes (0 added),|E| = 15 edges
Layer 4: M_4 = |V| = 3 nodes (0 added),|E| = 3 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 66 66
Number of Edge : 899 // 71
Layer 0: M_0 = |V| = 80 nodes (14 added),|E| = 899 edges
Layer 1: M_1 = |V| = 40 nodes (6 added),|E| = 368 edges
Layer 2: M_2 = |V| = 20 nodes (3 added),|E| = 119 edges
Layer 3: M_3 = |V| = 10 nodes (1 added),|E| = 35 edges
Layer 4: M_4 = |V| = 5 nodes (0 added),|E| = 10 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 69 69
Number of Edge : 1154 // 72
Layer 0: M_0 = |V| = 80 nodes (11 added),|E| = 1154 edges
Layer 1: M_1 = |V| = 40 nodes (5 added),|E| = 461 edges
Layer 2: M_2 = |V| = 20 nodes (2 added),|E| = 146 edges
Layer 3: M_3 = |V| = 10 nodes (1 added),|E| = 36 edges
Layer 4: M_

PPI Shape : 71 71
Number of Edge : 1173 // 93
Layer 0: M_0 = |V| = 80 nodes (9 added),|E| = 1173 edges
Layer 1: M_1 = |V| = 40 nodes (4 added),|E| = 461 edges
Layer 2: M_2 = |V| = 20 nodes (2 added),|E| = 138 edges
Layer 3: M_3 = |V| = 10 nodes (1 added),|E| = 36 edges
Layer 4: M_4 = |V| = 5 nodes (0 added),|E| = 10 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 89 89
Number of Edge : 2125 // 94
Layer 0: M_0 = |V| = 96 nodes (7 added),|E| = 2125 edges
Layer 1: M_1 = |V| = 48 nodes (3 added),|E| = 797 edges
Layer 2: M_2 = |V| = 24 nodes (1 added),|E| = 236 edges
Layer 3: M_3 = |V| = 12 nodes (0 added),|E| = 66 edges
Layer 4: M_4 = |V| = 6 nodes (0 added),|E| = 15 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 22 22
Number of Edge : 66 // 95
Layer 0: M_0 = |V| = 48 nodes (26 added),|E| = 66 edges
Layer 1: M_1 = |V| = 24 nodes (12 added),|E| = 28 edges
Layer 2: M_2 = |V| = 12 nodes (5 added),|E| = 13 edges
Layer 3: M_3 = |V| = 6 nodes (2 added),|E| = 5 edges
Layer 4: M_4

...

PPI Shape : 43 43
Number of Edge : 368 // 203
Layer 0: M_0 = |V| = 48 nodes (5 added),|E| = 368 edges
Layer 1: M_1 = |V| = 24 nodes (2 added),|E| = 135 edges
Layer 2: M_2 = |V| = 12 nodes (1 added),|E| = 46 edges
Layer 3: M_3 = |V| = 6 nodes (0 added),|E| = 14 edges
Layer 4: M_4 = |V| = 3 nodes (0 added),|E| = 3 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 109 109
Number of Edge : 2140 // 204
Layer 0: M_0 = |V| = 112 nodes (3 added),|E| = 2140 edges
Layer 1: M_1 = |V| = 56 nodes (0 added),|E| = 931 edges
Layer 2: M_2 = |V| = 28 nodes (0 added),|E| = 312 edges
Layer 3: M_3 = |V| = 14 nodes (0 added),|E| = 91 edges
Layer 4: M_4 = |V| = 7 nodes (0 added),|E| = 21 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 113 113
Number of Edge : 2396 // 205
Layer 0: M_0 = |V| = 128 nodes (15 added),|E| = 2396 edges
Layer 1: M_1 = |V| = 64 nodes (7 added),|E| = 1092 edges
Layer 2: M_2 = |V| = 32 nodes (3 added),|E| = 362 edges
Layer 3: M_3 = |V| = 16 nodes (1 added),|E| = 105 edges

../lib/coarsening.py:152: RuntimeWarning: divide by zero encountered in true_divide
  tval = vv[rs+jj] * (1.0/weights[tid] + 1.0/weights[nid])


PPI Shape : 66 66
Number of Edge : 1260 // 209
Layer 0: M_0 = |V| = 80 nodes (14 added),|E| = 1260 edges
Layer 1: M_1 = |V| = 40 nodes (7 added),|E| = 461 edges
Layer 2: M_2 = |V| = 20 nodes (3 added),|E| = 132 edges
Layer 3: M_3 = |V| = 10 nodes (1 added),|E| = 36 edges
Layer 4: M_4 = |V| = 5 nodes (0 added),|E| = 10 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 45 45
Number of Edge : 607 // 210
Layer 0: M_0 = |V| = 48 nodes (3 added),|E| = 607 edges
Layer 1: M_1 = |V| = 24 nodes (1 added),|E| = 228 edges
Layer 2: M_2 = |V| = 12 nodes (0 added),|E| = 66 edges
Layer 3: M_3 = |V| = 6 nodes (0 added),|E| = 15 edges
Layer 4: M_4 = |V| = 3 nodes (0 added),|E| = 3 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 58 58
Number of Edge : 921 // 211
Layer 0: M_0 = |V| = 64 nodes (6 added),|E| = 921 edges
Layer 1: M_1 = |V| = 32 nodes (3 added),|E| = 337 edges
Layer 2: M_2 = |V| = 16 nodes (1 added),|E| = 101 edges
Layer 3: M_3 = |V| = 8 nodes (0 added),|E| = 28 edges
Layer 4: M

PPI Shape : 86 86
Number of Edge : 1652 // 232
Layer 0: M_0 = |V| = 96 nodes (10 added),|E| = 1652 edges
Layer 1: M_1 = |V| = 48 nodes (5 added),|E| = 746 edges
Layer 2: M_2 = |V| = 24 nodes (2 added),|E| = 223 edges
Layer 3: M_3 = |V| = 12 nodes (1 added),|E| = 55 edges
Layer 4: M_4 = |V| = 6 nodes (0 added),|E| = 15 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 27 27
Number of Edge : 277 // 233
Layer 0: M_0 = |V| = 32 nodes (5 added),|E| = 277 edges
Layer 1: M_1 = |V| = 16 nodes (2 added),|E| = 90 edges
Layer 2: M_2 = |V| = 8 nodes (1 added),|E| = 21 edges
Layer 3: M_3 = |V| = 4 nodes (0 added),|E| = 6 edges
Layer 4: M_4 = |V| = 2 nodes (0 added),|E| = 1 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 119 119
Number of Edge : 4118 // 234
Layer 0: M_0 = |V| = 128 nodes (9 added),|E| = 4118 edges
Layer 1: M_1 = |V| = 64 nodes (4 added),|E| = 1513 edges
Layer 2: M_2 = |V| = 32 nodes (2 added),|E| = 420 edges
Layer 3: M_3 = |V| = 16 nodes (1 added),|E| = 105 edges
Layer

PPI Shape : 46 46
Number of Edge : 749 // 255
Layer 0: M_0 = |V| = 48 nodes (2 added),|E| = 749 edges
Layer 1: M_1 = |V| = 24 nodes (1 added),|E| = 241 edges
Layer 2: M_2 = |V| = 12 nodes (0 added),|E| = 66 edges
Layer 3: M_3 = |V| = 6 nodes (0 added),|E| = 15 edges
Layer 4: M_4 = |V| = 3 nodes (0 added),|E| = 3 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 21 21
Number of Edge : 184 // 256
Layer 0: M_0 = |V| = 32 nodes (11 added),|E| = 184 edges
Layer 1: M_1 = |V| = 16 nodes (5 added),|E| = 54 edges
Layer 2: M_2 = |V| = 8 nodes (2 added),|E| = 15 edges
Layer 3: M_3 = |V| = 4 nodes (1 added),|E| = 3 edges
Layer 4: M_4 = |V| = 2 nodes (0 added),|E| = 1 edges
<class 'scipy.sparse.coo.coo_matrix'>
PPI Shape : 37 37
Number of Edge : 512 // 257
Layer 0: M_0 = |V| = 48 nodes (11 added),|E| = 512 edges
Layer 1: M_1 = |V| = 24 nodes (5 added),|E| = 161 edges
Layer 2: M_2 = |V| = 12 nodes (2 added),|E| = 44 edges
Layer 3: M_3 = |V| = 6 nodes (1 added),|E| = 10 edges
Layer 4: M_4 = |V|

In [6]:
data_IC50 = pd.read_csv("Data/Table_S6_GDSC_Drug_response_IC50.csv")
data_IC50.drop(['Unnamed: 0'], axis='columns', inplace=True)
print(data_IC50.shape)

(734, 201)


In [9]:
dataset

{'exp': [array([[1.89289049, 1.67111535, 1.95871214, ..., 2.30112996, 1.99677152,
          2.11636866],
         [1.61266879, 1.65550187, 1.33247929, ..., 2.13777303, 1.84218341,
          2.0458107 ],
         [1.50915218, 1.78275987, 1.75343845, ..., 2.13428864, 1.65890547,
          2.23108801],
         ...,
         [1.6762205 , 1.86097231, 1.62675013, ..., 2.31916313, 1.94578817,
          2.13534807],
         [1.84295475, 1.84448402, 1.66741383, ..., 2.18200086, 2.03992909,
          1.90214131],
         [2.2233432 , 2.08044262, 1.36647275, ..., 1.68375478, 1.26557088,
          1.76192828]]),
  array([[1.59184464, 2.26276306, 1.03763924, ..., 1.57154316, 2.28150069,
          2.25919521],
         [1.29027572, 2.24025676, 1.15131768, ..., 1.407846  , 2.40356286,
          2.04253636],
         [1.78337258, 2.16315904, 1.00229431, ..., 1.3326172 , 2.13915876,
          2.0852583 ],
         ...,
         [1.19548372, 2.33903717, 1.15100046, ..., 1.50165517, 2.20060132,
      

In [ ]:
from scipy.sparse import coo_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats
n_fold = 3
Y_train, Y_test = train_test_split(data_IC50,  test_size=0.25, shuffle=True, random_state=20)


for cv in range(n_fold):
    Y_pred = np.zeros([Y_test.shape[0], Y_test.shape[1]])
    Y_test = np.zeros([Y_test.shape[0], Y_test.shape[1]])
#    print('n_fold:{}'.format(cv))
    j = 0
    for i in range(201):
        label = data_IC50.iloc[:,i]
        label = np.array(label)
        data_minmax = label[~np.isnan(label)]
        min = data_minmax.min()
        max = data_minmax.max()
        label = (label - min) / (max - min)
#         label_indexes = np.argwhere([~np.isnan(label)])[:, 1]
        
        train_idx, test_idx = train_test_split(range(label.shape[0]), test_size=0.25, shuffle=True, random_state=20)
        train_idx = np.array(train_idx)
        
        train_idx = train_idx[~np.isnan(label[train_idx])]        
        list_train, list_val = Validation(n_fold,train_idx)

        val_idx = train_idx[list_val[cv]]
        train_idx = train_idx[list_train[cv]]
        test_idx = np.array(test_idx)
        
        train_labels = label[train_idx]
        val_labels = label[val_idx]
        test_labels = label[test_idx]
        
        
        common = {}
        common['num_epochs']     = 40
        common['batch_size']     = 4
        common['decay_steps']    = train_idx.shape[0] / common['batch_size']
        common['eval_frequency'] = 10 * common['num_epochs']
        common['brelu']          = 'b1relu'
        common['pool']           = 'mpool1'

        common['regularization'] = 0
        common['dropout']        = 1
        common['learning_rate']  = 0.02
        common['decay_rate']     = 0.95
        common['momentum']       = 0.9
        common['F']              = [40]
        common['K']              = [40]
        common['p']              = [2]
        common['M']              = [1]
        # concatnate fully connected NN
        common['M1']             = [1]

        train_dataset = []
        val_dataset = []
        test_dataset = []
        for kegg_id in range(len(dataset['L'])):
            exp = dataset['exp'][kegg_id]
            perm = dataset['perm'][kegg_id]
#            print('kegg_id: {}, #nodes: {}'.format(kegg_id, len(perm)))
#            print(exp.shape)
#            print(train_idx.shape)
            train_data = exp[train_idx, :]
            val_data = exp[val_idx, :]
            test_data = exp[test_idx, :]
            train_data = coarsening.perm_data(train_data, perm)
            val_data = coarsening.perm_data(val_data, perm)
            test_data = coarsening.perm_data(test_data, perm)
            train_dataset.append(train_data)
            val_dataset.append(val_data)
            test_dataset.append(test_data)
#            print(train_data.shape, val_data.shape, test_data.shape)
        if True:
            name = 'cgconv_softmax'
            params = common.copy()
       
        model = models_update.cgcnn(dataset['L'], **params)
        loss, t_step = model.fit(train_dataset, train_labels, val_dataset, val_labels)


        Y_pred[:, j] = model.predict(test_dataset)
        Y_test[:, j] = test_labels        
#        print(Y_pred[:, j])
#        print(Y_test[:, j])
        j = j+1
#        np.savez(('4040.cv_{}'.format(cv)), Y_true=Y_test, Y_pred=Y_pred)

    np.savez(('4040.cv_{}'.format(cv)), Y_true=Y_test, Y_pred=Y_pred)


--------------------------------------
---0_th KEGG GCN ---
  input: M_0 = 48
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 48 * 40 / 2 = 960
    weights: F_0 * F_1 * K_1 = 1 * 40 * 40 = 1600
    biases: F_1 = 40
  layer 2: logits (softmax)
    representation: M_2 = 1
    weights: M_1 * M_2 = 960 * 1 = 960
    biases: M_2 = 1
---1_th KEGG GCN ---
  input: M_0 = 32
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 32 * 40 / 2 = 640
    weights: F_0 * F_1 * K_1 = 1 * 40 * 40 = 1600
    biases: F_1 = 40
  layer 2: logits (softmax)
    representation: M_2 = 1
    weights: M_1 * M_2 = 640 * 1 = 640
    biases: M_2 = 1
---2_th KEGG GCN ---
  input: M_0 = 32
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 32 * 40 / 2 = 640
    weights: F_0 * F_1 * K_1 = 1 * 40 * 40 = 1600
    biases: F_1 = 40
  layer 2: logits (softmax)
    representation: M_2 = 1
    weights: M_1 * M_2 = 640 * 1 = 640
    biases: M_2 = 1
---3_th KEGG GCN ---
  input: M_0 = 32
  layer 1: cgconv1
   

step 400 / 5430 (epoch 2.95 / 40):
  learning_rate = 1.80e-02, loss_average = 4.55e-02
  validation loss: 4.22e-02
  time: 1288s (wall 445s)
step 800 / 5430 (epoch 5.89 / 40):
  learning_rate = 1.55e-02, loss_average = 3.12e-02
  validation loss: 3.40e-02
  time: 2449s (wall 745s)
step 1200 / 5430 (epoch 8.84 / 40):
  learning_rate = 1.33e-02, loss_average = 3.69e-02
  validation loss: 4.05e-02
  time: 3437s (wall 876s)
step 1600 / 5430 (epoch 11.79 / 40):
  learning_rate = 1.14e-02, loss_average = 3.09e-02
  validation loss: 3.39e-02
  time: 4429s (wall 1008s)
step 2000 / 5430 (epoch 14.73 / 40):
  learning_rate = 9.75e-03, loss_average = 2.98e-02
  validation loss: 3.68e-02
  time: 5411s (wall 1136s)
step 2400 / 5430 (epoch 17.68 / 40):
  learning_rate = 8.36e-03, loss_average = 3.72e-02
  validation loss: 3.21e-02
  time: 6401s (wall 1268s)
step 2800 / 5430 (epoch 20.63 / 40):
  learning_rate = 7.17e-03, loss_average = 2.67e-02
  validation loss: 3.14e-02
  time: 7361s (wall 1392s)


    weights: M_1 * M_2 = 640 * 1 = 640
    biases: M_2 = 1
---28_th KEGG GCN ---
  input: M_0 = 32
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 32 * 40 / 2 = 640
    weights: F_0 * F_1 * K_1 = 1 * 40 * 40 = 1600
    biases: F_1 = 40
  layer 2: logits (softmax)
    representation: M_2 = 1
    weights: M_1 * M_2 = 640 * 1 = 640
    biases: M_2 = 1
---29_th KEGG GCN ---
  input: M_0 = 32
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 32 * 40 / 2 = 640
    weights: F_0 * F_1 * K_1 = 1 * 40 * 40 = 1600
    biases: F_1 = 40
  layer 2: logits (softmax)
    representation: M_2 = 1
    weights: M_1 * M_2 = 640 * 1 = 640
    biases: M_2 = 1
---30_th KEGG GCN ---
  input: M_0 = 32
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 32 * 40 / 2 = 640
    weights: F_0 * F_1 * K_1 = 1 * 40 * 40 = 1600
    biases: F_1 = 40
  layer 2: logits (softmax)
    representation: M_2 = 1
    weights: M_1 * M_2 = 640 * 1 = 640
    biases: M_2 = 1
---31_th KEGG GCN ---
  input: M_0 = 3

step 400 / 5440 (epoch 2.94 / 40):
  learning_rate = 1.80e-02, loss_average = 2.38e-02
  validation loss: 2.32e-02
  time: 1190s (wall 417s)
step 800 / 5440 (epoch 5.88 / 40):
  learning_rate = 1.55e-02, loss_average = 1.70e-02
  validation loss: 1.83e-02
  time: 2245s (wall 698s)
step 1200 / 5440 (epoch 8.82 / 40):
  learning_rate = 1.33e-02, loss_average = 2.00e-02
  validation loss: 1.75e-02
  time: 3124s (wall 803s)
step 1600 / 5440 (epoch 11.76 / 40):
  learning_rate = 1.14e-02, loss_average = 2.33e-02
  validation loss: 2.57e-02
  time: 4003s (wall 908s)
step 2000 / 5440 (epoch 14.71 / 40):
  learning_rate = 9.75e-03, loss_average = 1.83e-02
  validation loss: 1.98e-02
  time: 4883s (wall 1013s)
step 2400 / 5440 (epoch 17.65 / 40):
  learning_rate = 8.36e-03, loss_average = 1.63e-02
  validation loss: 1.68e-02
  time: 5761s (wall 1118s)
step 2800 / 5440 (epoch 20.59 / 40):
  learning_rate = 7.17e-03, loss_average = 1.86e-02
  validation loss: 2.56e-02
  time: 6640s (wall 1223s)
s

    weights: F_0 * F_1 * K_1 = 1 * 40 * 40 = 1600
    biases: F_1 = 40
  layer 2: logits (softmax)
    representation: M_2 = 1
    weights: M_1 * M_2 = 640 * 1 = 640
    biases: M_2 = 1
---18_th KEGG GCN ---
  input: M_0 = 32
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 32 * 40 / 2 = 640
    weights: F_0 * F_1 * K_1 = 1 * 40 * 40 = 1600
    biases: F_1 = 40
  layer 2: logits (softmax)
    representation: M_2 = 1
    weights: M_1 * M_2 = 640 * 1 = 640
    biases: M_2 = 1
---19_th KEGG GCN ---
  input: M_0 = 32
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 32 * 40 / 2 = 640
    weights: F_0 * F_1 * K_1 = 1 * 40 * 40 = 1600
    biases: F_1 = 40
  layer 2: logits (softmax)
    representation: M_2 = 1
    weights: M_1 * M_2 = 640 * 1 = 640
    biases: M_2 = 1
---20_th KEGG GCN ---
  input: M_0 = 32
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 32 * 40 / 2 = 640
    weights: F_0 * F_1 * K_1 = 1 * 40 * 40 = 1600
    biases: F_1 = 40
  layer 2: logits (softmax

step 400 / 5270 (epoch 3.04 / 40):
  learning_rate = 1.71e-02, loss_average = 3.17e-02
  validation loss: 3.59e-02
  time: 1196s (wall 437s)
step 800 / 5270 (epoch 6.07 / 40):
  learning_rate = 1.47e-02, loss_average = 3.12e-02
  validation loss: 4.43e-02
  time: 2243s (wall 724s)
step 1200 / 5270 (epoch 9.11 / 40):
  learning_rate = 1.26e-02, loss_average = 3.48e-02
  validation loss: 3.39e-02
  time: 3115s (wall 840s)
step 1600 / 5270 (epoch 12.14 / 40):
  learning_rate = 1.08e-02, loss_average = 4.49e-02
  validation loss: 3.49e-02
  time: 3989s (wall 956s)
step 2000 / 5270 (epoch 15.18 / 40):
  learning_rate = 9.27e-03, loss_average = 4.47e-02
  validation loss: 3.37e-02
  time: 4950s (wall 1093s)
step 2400 / 5270 (epoch 18.22 / 40):
  learning_rate = 7.94e-03, loss_average = 3.65e-02
  validation loss: 3.21e-02
  time: 5910s (wall 1230s)
step 2800 / 5270 (epoch 21.25 / 40):
  learning_rate = 6.81e-03, loss_average = 3.91e-02
  validation loss: 3.15e-02
  time: 6875s (wall 1368s)
s

step 400 / 5280 (epoch 3.03 / 40):
  learning_rate = 1.71e-02, loss_average = 5.64e-02
  validation loss: 4.67e-02
  time: 1280s (wall 450s)
step 800 / 5280 (epoch 6.06 / 40):
  learning_rate = 1.47e-02, loss_average = 4.62e-02
  validation loss: 4.96e-02
  time: 2439s (wall 763s)
step 1200 / 5280 (epoch 9.09 / 40):
  learning_rate = 1.26e-02, loss_average = 4.32e-02
  validation loss: 3.34e-02
  time: 3423s (wall 898s)
step 1600 / 5280 (epoch 12.12 / 40):
  learning_rate = 1.08e-02, loss_average = 4.25e-02
  validation loss: 3.14e-02
  time: 4408s (wall 1033s)
step 2000 / 5280 (epoch 15.15 / 40):
  learning_rate = 9.27e-03, loss_average = 4.61e-02
  validation loss: 3.20e-02
  time: 5391s (wall 1167s)
step 2400 / 5280 (epoch 18.18 / 40):
  learning_rate = 7.94e-03, loss_average = 4.25e-02
  validation loss: 3.12e-02
  time: 6377s (wall 1302s)
step 2800 / 5280 (epoch 21.21 / 40):
  learning_rate = 6.81e-03, loss_average = 2.91e-02
  validation loss: 2.99e-02
  time: 7360s (wall 1436s)


    representation: M_0 * F_1 / p_1 = 32 * 40 / 2 = 640
    weights: F_0 * F_1 * K_1 = 1 * 40 * 40 = 1600
    biases: F_1 = 40
  layer 2: logits (softmax)
    representation: M_2 = 1
    weights: M_1 * M_2 = 640 * 1 = 640
    biases: M_2 = 1
---18_th KEGG GCN ---
  input: M_0 = 32
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 32 * 40 / 2 = 640
    weights: F_0 * F_1 * K_1 = 1 * 40 * 40 = 1600
    biases: F_1 = 40
  layer 2: logits (softmax)
    representation: M_2 = 1
    weights: M_1 * M_2 = 640 * 1 = 640
    biases: M_2 = 1
---19_th KEGG GCN ---
  input: M_0 = 32
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 32 * 40 / 2 = 640
    weights: F_0 * F_1 * K_1 = 1 * 40 * 40 = 1600
    biases: F_1 = 40
  layer 2: logits (softmax)
    representation: M_2 = 1
    weights: M_1 * M_2 = 640 * 1 = 640
    biases: M_2 = 1
---20_th KEGG GCN ---
  input: M_0 = 32
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 32 * 40 / 2 = 640
    weights: F_0 * F_1 * K_1 = 1 * 40 * 4

step 400 / 5210 (epoch 3.07 / 40):
  learning_rate = 1.71e-02, loss_average = 5.08e-02
  validation loss: 4.97e-02
  time: 1281s (wall 463s)
step 800 / 5210 (epoch 6.14 / 40):
  learning_rate = 1.47e-02, loss_average = 5.42e-02
  validation loss: 4.63e-02
  time: 2429s (wall 776s)
step 1200 / 5210 (epoch 9.21 / 40):
  learning_rate = 1.26e-02, loss_average = 5.29e-02
  validation loss: 4.48e-02
  time: 3400s (wall 916s)
step 1600 / 5210 (epoch 12.28 / 40):
  learning_rate = 1.08e-02, loss_average = 4.56e-02
  validation loss: 6.42e-02
  time: 4374s (wall 1056s)
step 2000 / 5210 (epoch 15.36 / 40):
  learning_rate = 9.27e-03, loss_average = 5.79e-02
  validation loss: 4.36e-02
  time: 5345s (wall 1196s)


In [47]:
from scipy import stats
RMSE = []
PCC = []
SCC = []

for i in range(4):
    y_test = Y_test[:, i]
    y_pred = Y_pred[:, i]
    y_pred = y_pred[~np.isnan(y_test)]
    y_test = y_test[~np.isnan(y_test)]

    RMSE.append(mean_squared_error(y_test,y_pred))
    PCC.append(stats.pearsonr(y_test,y_pred)[0])
    SCC.append(stats.spearmanr(y_test,y_pred)[0])
print("RMSE :",RMSE)
print("PCC :",PCC)
print("SCC :",SCC)
# 50 40 2 1 1

RMSE : [0.03449682940019994, 0.016506825518722328, 0.03304470752500458, 0.0359634220979898]
PCC : [0.3829299218333685, 0.41952147256776295, 0.23990148477324408, 0.27363753511941785]
SCC : [0.3694496743726658, 0.4427821974239871, 0.23844460592236225, 0.2894369167772668]


In [12]:
from scipy import stats 
RMSE = []
PCC = []
SCC = []

for i in range(4):
    y_test = Y_test[:, i]
    y_pred = Y_pred[:, i]
    y_pred = y_pred[~np.isnan(y_test)]
    y_test = y_test[~np.isnan(y_test)]

    RMSE.append(mean_squared_error(y_test,y_pred))
    PCC.append(stats.pearsonr(y_test,y_pred)[0])
    SCC.append(stats.spearmanr(y_test,y_pred)[0])
print("RMSE :",RMSE)
print("PCC :",PCC)
print("SCC :",SCC)

## 40 40 2 1 1 ( 1, 2) #따봉

RMSE : [0.0, 0.0, 0.0, 0.0]
PCC : [nan, nan, nan, nan]
SCC : [nan, nan, nan, nan]


/home/bmlserver/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/bmlserver/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/bmlserver/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/bmlserver/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/bmlserver/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/home/bmlserver/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.p

In [ ]:
from scipy.sparse import coo_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats
U = [137,195,172,55]
n_fold = 3
C = [0,1,2,3]
Y_train, Y_test = train_test_split(data_IC50,  test_size=0.25, shuffle=True, random_state=20)
#print(Y_test.shape)

Kernel_Num =  [[10, 10, 10], [20, 20, 20]]
Kernel_Size = [[5, 5, 5], [10, 10, 10], [30, 30, 30]]
for ker_num in range(len(Kernel_Num)):
    for ker_size in range(len(Kernel_Size)):
        for cv in range(1):
            Y_pred = np.zeros([Y_test.shape[0], Y_test.shape[1]])
            Y_test = np.zeros([Y_test.shape[0], Y_test.shape[1]])
        #    print('n_fold:{}'.format(cv))
            j = 0
            for i in C:
                label = data_IC50.iloc[:,i]
                label = np.array(label)
                data_minmax = label[~np.isnan(label)]
                min = data_minmax.min()
                max = data_minmax.max()
                label = (label - min) / (max - min)
        #         label_indexes = np.argwhere([~np.isnan(label)])[:, 1]

                train_idx, test_idx = train_test_split(range(label.shape[0]), test_size=0.25, shuffle=True, random_state=20)
                train_idx = np.array(train_idx)

                train_idx = train_idx[~np.isnan(label[train_idx])]        
                list_train, list_val = Validation(n_fold,train_idx)

                val_idx = train_idx[list_val[cv]]
                train_idx = train_idx
                test_idx = np.array(test_idx)

                train_labels = label[train_idx]
                val_labels = label[val_idx]
                test_labels = label[test_idx]


                common = {}
                common['num_epochs']     = 40
                common['batch_size']     = 4
                common['decay_steps']    = train_idx.shape[0] / common['batch_size']
                common['eval_frequency'] = 10 * common['num_epochs']
                common['brelu']          = 'b1relu'
                common['pool']           = 'mpool1'

                common['regularization'] = 0
                common['dropout']        = 1
                common['learning_rate']  = 0.02
                common['decay_rate']     = 0.95
                common['momentum']       = 0.9
                common['F']              = Kernel_Num[ker_num]
                common['K']              = Kernel_Size[ker_size]
                common['p']              = [2,2,2]
                common['M']              = [1]
                # concatnate fully connected NN
                common['M1']             = [1]

                train_dataset = []
                val_dataset = []
                test_dataset = []
                for kegg_id in range(len(dataset['L'])):
                    exp = dataset['exp'][kegg_id]
                    perm = dataset['perm'][kegg_id]
        #            print('kegg_id: {}, #nodes: {}'.format(kegg_id, len(perm)))
        #            print(exp.shape)
        #            print(train_idx.shape)
                    train_data = exp[train_idx, :]
                    val_data = exp[val_idx, :]
                    test_data = exp[test_idx, :]
                    train_data = coarsening.perm_data(train_data, perm)
                    val_data = coarsening.perm_data(val_data, perm)
                    test_data = coarsening.perm_data(test_data, perm)
                    train_dataset.append(train_data)
                    val_dataset.append(val_data)
                    test_dataset.append(test_data)
        #            print(train_data.shape, val_data.shape, test_data.shape)
                if True:
                    name = 'cgconv_softmax'
                    params = common.copy()

                model = models_update.cgcnn(dataset['L'], **params)
                loss, t_step = model.fit(train_dataset, train_labels, val_dataset, val_labels)


                Y_pred[:, j] = model.predict(test_dataset)
                Y_test[:, j] = test_labels        
                print(Y_pred[:, j])
                print(Y_test[:, j])
                j = j+1
                np.savez(('Kernel_Test_{}_{}'.format(Kernel_Num[ker_num],Kernel_Size[ker_size])), Y_true=Y_test, Y_pred=Y_pred)

            


--------------------------------------
---0_th KEGG GCN ---
  input: M_0 = 48
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 48 * 10 / 2 = 240
    weights: F_0 * F_1 * K_1 = 1 * 10 * 5 = 50
    biases: F_1 = 10
  layer 2: cgconv2
    representation: M_1 * F_2 / p_2 = 24 * 10 / 2 = 120
    weights: F_1 * F_2 * K_2 = 10 * 10 * 5 = 500
    biases: F_2 = 10
  layer 3: cgconv3
    representation: M_2 * F_3 / p_3 = 12 * 10 / 2 = 60
    weights: F_2 * F_3 * K_3 = 10 * 10 * 5 = 500
    biases: F_3 = 10
  layer 4: logits (softmax)
    representation: M_4 = 1
    weights: M_3 * M_4 = 60 * 1 = 60
    biases: M_4 = 1
---1_th KEGG GCN ---
  input: M_0 = 32
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 32 * 10 / 2 = 160
    weights: F_0 * F_1 * K_1 = 1 * 10 * 5 = 50
    biases: F_1 = 10
  layer 2: cgconv2
    representation: M_1 * F_2 / p_2 = 16 * 10 / 2 = 80
    weights: F_1 * F_2 * K_2 = 10 * 10 * 5 = 500
    biases: F_2 = 10
  layer 3: cgconv3
    representation: M_2 * F_3 / 

    representation: M_0 * F_1 / p_1 = 80 * 10 / 2 = 400
    weights: F_0 * F_1 * K_1 = 1 * 10 * 5 = 50
    biases: F_1 = 10
  layer 2: cgconv2
    representation: M_1 * F_2 / p_2 = 40 * 10 / 2 = 200
    weights: F_1 * F_2 * K_2 = 10 * 10 * 5 = 500
    biases: F_2 = 10
  layer 3: cgconv3
    representation: M_2 * F_3 / p_3 = 20 * 10 / 2 = 100
    weights: F_2 * F_3 * K_3 = 10 * 10 * 5 = 500
    biases: F_3 = 10
  layer 4: logits (softmax)
    representation: M_4 = 1
    weights: M_3 * M_4 = 100 * 1 = 100
    biases: M_4 = 1
---139_th KEGG GCN ---
  input: M_0 = 32
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 32 * 10 / 2 = 160
    weights: F_0 * F_1 * K_1 = 1 * 10 * 5 = 50
    biases: F_1 = 10
  layer 2: cgconv2
    representation: M_1 * F_2 / p_2 = 16 * 10 / 2 = 80
    weights: F_1 * F_2 * K_2 = 10 * 10 * 5 = 500
    biases: F_2 = 10
  layer 3: cgconv3
    representation: M_2 * F_3 / p_3 = 8 * 10 / 2 = 40
    weights: F_2 * F_3 * K_3 = 10 * 10 * 5 = 500
    biases: F_3 = 10


step 400 / 5430 (epoch 2.95 / 40):
  learning_rate = 1.80e-02, loss_average = 3.09e-02
  validation loss: 3.34e-02
  time: 1008s (wall 273s)
step 800 / 5430 (epoch 5.89 / 40):
  learning_rate = 1.55e-02, loss_average = 3.46e-02
  validation loss: 3.51e-02
  time: 1965s (wall 490s)
step 1200 / 5430 (epoch 8.84 / 40):
  learning_rate = 1.33e-02, loss_average = 3.44e-02
  validation loss: 4.35e-02
  time: 2809s (wall 598s)
step 1600 / 5430 (epoch 11.79 / 40):
  learning_rate = 1.14e-02, loss_average = 4.54e-02
  validation loss: 3.78e-02
  time: 3653s (wall 705s)
step 2000 / 5430 (epoch 14.73 / 40):
  learning_rate = 9.75e-03, loss_average = 2.97e-02
  validation loss: 3.35e-02
  time: 4497s (wall 813s)
step 2400 / 5430 (epoch 17.68 / 40):
  learning_rate = 8.36e-03, loss_average = 2.25e-02
  validation loss: 3.46e-02
  time: 5343s (wall 921s)
step 2800 / 5430 (epoch 20.63 / 40):
  learning_rate = 7.17e-03, loss_average = 3.10e-02
  validation loss: 3.31e-02
  time: 6187s (wall 1029s)
ste

In [10]:
FCa = [[1], [16,1], [128,1]]
FCb = [[1], [16,1], [128,1]]
for FC1 in range(len(FCa)):
    for FC2 in range(len(FCb)):
        print("Test_{}_{}".format(FCa[FC1],FCb[FC2]))

Test_[1]_[1]
Test_[1]_[16, 1]
Test_[1]_[128, 1]
Test_[16, 1]_[1]
Test_[16, 1]_[16, 1]
Test_[16, 1]_[128, 1]
Test_[128, 1]_[1]
Test_[128, 1]_[16, 1]
Test_[128, 1]_[128, 1]


In [14]:
Kernel_Num =  [[10, 10, 10], [20, 20, 20]]
Kernel_Size = [[5, 5, 5], [10, 10, 10], [30, 30, 30]]
for ker_num in range(len(Kernel_Num)):
    for ker_size in range(len(Kernel_Size)):
        print(Kernel_Num[ker_num], Kernel_Size[ker_size])
        

[10, 10, 10] [5, 5, 5]
[10, 10, 10] [10, 10, 10]
[10, 10, 10] [30, 30, 30]
[20, 20, 20] [5, 5, 5]
[20, 20, 20] [10, 10, 10]
[20, 20, 20] [30, 30, 30]


In [1]:
import os
print(os.getpid())

2484738
